In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor

In [2]:
k=20
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

# tolerance setting
tol = 1e-6
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i.cmd' % k)
solverF.init_grid()
solverF.init_temperature_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFDSR%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp)
solverS.output_file = 'solid_outputs/resFDSR%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_robin_interface(tag=interface_tag)
solverS.init_solver()

In [6]:
# coupling data and setups
abTempF = InterfaceData(size=inodes, value=300.0)
abTempF2S = InterfaceData(size=inodes, value=300.0)
TempS = InterfaceData(size=inodes, value=T_bot)
tempF = InterfaceData(size=inodes, value=300.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=tol)

# maximum pc steps allowed
max_pc_steps = 200

# since this problem has no mesh motion and both kappa are material properties
# so the heat transfer coefficients don't vary in time
hF2S = solverF.get_heat_transfer_coeff(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)[f2s]

In [7]:
# coupling timing information, use fixed time step for now
Tfinal = 1.0
N = 100
dt = Tfinal/N
t = 0.0
avg_pc_iterations = 0
flog = open('FDSR%i.log' % k, mode='w')
flog.write('Fluid Dirichlet with solid Robin setting, study case: %i\n' % k)
flog.close()
flog = open('FDSR%i.log' % k, mode='a')

In [8]:
# begin to couple
for step in range(N):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    while True:
        pc_counts = pc_counts + 1
        
        # back up previous interface value
        tempF.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid ambient temperature
        abTempF.curr[:] = solverF.get_ambient_temperature(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # retrieve the wall temperature for convergence testing
        tempF.curr[:] = solverF.get_temperature_interface(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # mapping
        abTempF2S.curr[:] = abTempF.curr[f2s]
        # update solid flux interface
        solverS.update_robin_interface(
            tag=interface_tag,
            Ta=abTempF2S.curr,
            h_coeff=hF2S
        )
        
        # advance solid
        solverS.advance(t=t, dt=dt)
        # retrieve solid interface temperature
        TempS.curr[:] = solverS.get_interface_temperature(tag=interface_tag)
        
        # mapping
        tempF.curr[:] = TempS.curr[s2f]
        
        # update residual
        tempF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(tempF)
        if is_conv or pc_counts >= max_pc_steps:
            solverF.put_temperature_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=tempF.curr
            )
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            under_relax.determine_omega(tempF)
            under_relax.update_solution(tempF)
            solverF.put_temperature_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=tempF.curr
            )
            solverF.restore_previous_solutions()
            solverS.restore()  

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    solverS.write()
    fluid_ofile.write(solverF, t)
    avg_pc_iterations = avg_pc_iterations + pc_counts
flog.close()

step=0, pc_iterations=7.
step=1, pc_iterations=1.
step=2, pc_iterations=1.
step=3, pc_iterations=1.
step=4, pc_iterations=1.
step=5, pc_iterations=1.
step=6, pc_iterations=1.
step=7, pc_iterations=1.
step=8, pc_iterations=1.
step=9, pc_iterations=1.
step=10, pc_iterations=1.
step=11, pc_iterations=1.
step=12, pc_iterations=1.
step=13, pc_iterations=1.
step=14, pc_iterations=1.
step=15, pc_iterations=1.
step=16, pc_iterations=1.
step=17, pc_iterations=1.
step=18, pc_iterations=1.
step=19, pc_iterations=1.
step=20, pc_iterations=1.
step=21, pc_iterations=1.
step=22, pc_iterations=1.
step=23, pc_iterations=1.
step=24, pc_iterations=1.
step=25, pc_iterations=1.
step=26, pc_iterations=1.
step=27, pc_iterations=1.
step=28, pc_iterations=1.
step=29, pc_iterations=1.
step=30, pc_iterations=1.
step=31, pc_iterations=1.
step=32, pc_iterations=1.
step=33, pc_iterations=1.
step=34, pc_iterations=1.
step=35, pc_iterations=1.
step=36, pc_iterations=1.
step=37, pc_iterations=1.
step=38, pc_iterations

In [9]:
theta=(solverF.get_temperature_interface(3,0,1)-300)/10
print(theta)
sio.savemat('numeric_res/kFDSR%i.mat' % k, {'theta': theta, 'avg_pc_its': avg_pc_iterations/N})

[0.9104463  0.91205811 0.91458167 0.9169817  0.91920201 0.92115103
 0.92293414 0.92454659 0.92602683 0.9274304  0.92871269 0.9299159
 0.93103127 0.93208265 0.93305961 0.93397979 0.93483896 0.93565206
 0.93641339 0.93712619 0.93779716 0.93842591 0.93901937 0.93957412
 0.94009412 0.94058088 0.94104025 0.94146935 0.94187301 0.94225041
 0.94260245 0.94293297 0.94324093 0.94352815 0.94379798 0.94404771
 0.94428043 0.94449578 0.94469709 0.94488263 0.94505436 0.94521346
 0.94535891 0.94549236 0.94561425 0.94572494 0.94582513 0.94591496
 0.94599483 0.94606592 0.94612689 0.94617948 0.94622285 0.94625769
 0.94628438 0.94630203 0.94631237 0.94631373 0.94630712 0.94629203
 0.9462685  0.94623716 0.9461963  0.94614707 0.94608773 0.9460196
 0.94594179 0.94585291 0.94575411 0.94564347 0.94552063 0.94538608
 0.94523727 0.94507569 0.94489809 0.94470488 0.94449275 0.94426376
 0.94401474 0.94374306 0.94344961 0.9431288  0.9427809  0.9424056
 0.94199476 0.94154958 0.94106197 0.94053201 0.93995276 0.9393147